# Principal_component_analysis_AGDC_looped.ipynb

This code combines all of the individual datasets used previously into a principal component analysis. NDVI slope, Landsat data and band indices, elevation and gamma data are all extracted and added into the PCA. The first 6 principal components are plotted and then tested for statistical difference based on the palaeovalleys shape file. The significance of each principal component helps to determine which PC best captures palaeovalley location.

** Code dependencies **
- csv file containing the bounding boxes for the case study site/s
- palaeovalleys 2012 shape file
- Landsat band average netcdf files produced by "Extract_AGDC_for_study_sites_looped"
- NDVI slope netcdf files created by "Palaeovalley_NDVI_linear_regression_raijin_JanJun/JulDec.py

** Accompanying code**
- Princical_component_analysis_AGDC.ipynb - shows how this code works using a single example study site

Created by Claire Krause, Datacube v 1.1.17, python v3

Based on PCA code from Neil Symington

In [1]:
%matplotlib inline
import fiona
import shapely.geometry
import rasterio
import rasterio.features
import geopandas as gp
import datacube
datacube.set_options(reproject_threads=1)
dc = datacube.Datacube(app='PCA')
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import xarray as xr
import scipy.stats
import pandas
import csv
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale, Imputer

In [2]:
# Set up some functions to use later in the code
def warp_geometry(geom, src_crs, dst_crs):
    """
    warp geometry from src_crs to dst_crs
    """
    return shapely.geometry.shape(rasterio.warp.transform_geom(src_crs, dst_crs, shapely.geometry.mapping(geom)))

def geometry_mask(geom, geobox, all_touched=False, invert=False):
    """
    rasterize geometry into a binary mask where pixels that overlap geometry are False
    """
    return rasterio.features.geometry_mask([geom],
                                           out_shape=geobox.shape,
                                           transform=geobox.affine,
                                           all_touched=all_touched,
                                           invert=invert)
def write_to_csv(OUTPUT_path, row, idx):
    if idx == 0:
        with open(OUTPUT_path,'w') as csvFile:
            writer = csv.writer(csvFile)
            header = ['name', 'ttest', 'KS_test']
            writer.writerow(header)
            writer.writerow(row)
    else:
        with open(OUTPUT_path,'a') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(row)

def write_geotiff(filename, dataset, time_index=None, profile_override=None):
    """
    Write an xarray dataset to a geotiff
    :attr bands: ordered list of dataset names
    :attr time_index: time index to write to file
    :attr dataset: xarray dataset containing multiple bands to write to file
    :attr profile_override: option dict, overrides rasterio file creation options.
    """
    profile_override = profile_override or {}

    dtypes = {val.dtype for val in dataset.data_vars.values()}
    assert len(dtypes) == 1  # Check for multiple dtypes

    profile = DEFAULT_PROFILE.copy()
    profile.update({
        'width': dataset.dims['x'],
        'height': dataset.dims['y'],
        'affine': dataset.affine,
        #'crs': dataset.crs.crs_str,
        'crs': dataset.crs,
        'count': len(dataset.data_vars),
        'dtype': str(dtypes.pop())
    })
    profile.update(profile_override)

    with rasterio.open(filename, 'w', **profile) as dest:
        for bandnum, data in enumerate(dataset.data_vars.values(), start=1):
            #dest.write(data.isel(time=time_index).data, bandnum)
            dest.write(data, bandnum)
            print ('Done')

## Load in the case study areas and shape file


In [3]:
# Set up the case study bounding box (to make the file smaller and avoid memory errors)
# Read in a csv file with all case study bounding boxes
names = pandas.read_csv('/g/data/p25/cek156/case_study_sites_small.csv', delimiter = ',')

# Read in the palaeovalley 2012 shape file
shp = gp.GeoDataFrame.from_file('/g/data/p25/cek156/Palaeovalleys_2012.shp')

## Set up and run our PCA

In [ ]:
for num, site in enumerate(names.Name[:-6]):
    Studysite = names.ix[num]
    print ('Working on ' + Studysite.Name)
    
    ## Check for files so we don't have to run everything multiple times
    stats_check = False #os.path.isfile('/g/data/p25/cek156/' + site + '_PCA_stats.csv')
    if stats_check == True:
        print('PCA already done for ' + site)
    else:
        ###### Landsat ############
        # Set up our file names to be read in
        blue_mean = '/g/data/p25/cek156/' + site + '/' + site + '_blue_time_mean.nc'
        green_mean = '/g/data/p25/cek156/' + site + '/' + site + '_green_time_mean.nc'
        red_mean = '/g/data/p25/cek156/' + site + '/' + site + '_red_time_mean.nc'
        nir_mean = '/g/data/p25/cek156/' + site + '/' + site + '_nir_time_mean.nc'
        swir1_mean = '/g/data/p25/cek156/' + site + '/' + site + '_swir1_time_mean.nc'
        swir2_mean = '/g/data/p25/cek156/' + site + '/' + site + '_swir2_time_mean.nc'

        # We need to check that the mean files have been created before we try to read them in
        file_checkb = os.path.isfile(blue_mean)
        file_checkg = os.path.isfile(green_mean)
        file_checkr = os.path.isfile(red_mean)
        file_checkn = os.path.isfile(nir_mean)
        file_checks1 = os.path.isfile(swir1_mean)
        file_checks2 = os.path.isfile(swir2_mean)

        # If all the files are then, then we can read them in
        if file_checkb == True & file_checkg == True & file_checkr == True & file_checkn == True & file_checks1 == True & file_checks2 == True:
            blue = xr.open_dataset(blue_mean)
            green = xr.open_dataset(green_mean)
            red = xr.open_dataset(red_mean)
            nir = xr.open_dataset(nir_mean)
            swir1 = xr.open_dataset(swir1_mean) 
            swir2 = xr.open_dataset(swir2_mean)

            ########### Set up the analyses you would like to do ###########################
            # Check out http://www.indexdatabase.de/db/i.php for a list of a whole bunch of indices, 
            # as well as their platform specific formulas (if you click on the index name).
            analyses = {'only_blue':blue.blue,
            'only_green':green.green,
            'only_red':red.red,
            'only_nir':nir.nir,
            'only_swir1':swir1.swir1,
            'only_swir2':swir2.swir2,
            'greenness':(green.green / red.red),
            'drought':(swir2.swir2 / nir.nir),
            'ferrous':(swir1.swir1 / nir.nir),
            'clay':(swir1.swir1 / swir2.swir2),
            'soilBG':(nir.nir - (2.4 * red.red)), # Soil background line
            'soilComp':((swir1.swir1 - nir.nir) / (swir1.swir1 + nir.nir)), # Soil composition index
            'SAVI':(((nir.nir - red.red) / (nir.nir + red.red + 0.5))*(1 + 0.5)), #Soil adjusted vegetation index, where L = 0.5
            'FalseCol':((nir.nir + red.red + green.green)), #False colour
            'RealCol':((red.red + green.green + blue.blue)), #Real colour
            'NDMI':((nir.nir - swir1.swir1) / (nir.nir + swir1.swir1)), #normalised difference moisture index
            'NDSI':((swir1.swir1 - swir2.swir2) / (swir1.swir1 + swir2.swir2))} # normalised difference salinity index
            ################################################################################

            Landsat = xr.Dataset(analyses)

        ###### NDVI slope #############    
        # Grab the Jan - June data
        input_filename = '/g/data/p25/cek156/NDVI/' + Studysite.Name + '/' + Studysite.Name + '_NDVI_slope_JanJun.nc'
        NDVIdataJanJun = xr.open_dataset(input_filename)
        ## Grab the Jul - Dec data
        input_filename = '/g/data/p25/cek156/NDVI/' + Studysite.Name + '/' + Studysite.Name + '_NDVI_slope_JulDec.nc'
        NDVIdataJulDec = xr.open_dataset(input_filename)

        ###### Elevation ##########
        # Read in the elevation data from the datacube
        query = {'lat': (names.maxlat[num], names.minlat[num]), 
             'lon': (names.minlon[num], names.maxlon[num]),
             'resolution': (-250, 250), 'output_crs': 'EPSG:3577'}
        elev = dc.load(product = 'srtm_dem1sv1_0', group_by='solar_day', **query)
        elev = elev.dem_s
        elev = elev.squeeze()

        ###### Gamma ##########
        # Read in the data from the datacube
        query = {'lat': (names.maxlat[num], names.minlat[num]), 
           'lon': (names.minlon[num], names.maxlon[num]),
            'resolution': (-250, 250), 'output_crs': 'EPSG:3577'}
        Gamma = dc.load(product = 'gamma_ray', group_by='solar_day', **query)

        ###### Merge all the data together #############
        all_data = Landsat.copy()
        all_data['elev'] = elev
        all_data['NDVIJJ'] = NDVIdataJanJun.slope
        all_data['NDVIJD'] = NDVIdataJulDec.slope

        all_data = all_data.merge(Gamma)
        all_data = all_data.squeeze()

        for i, items in enumerate(all_data.data_vars.keys()):
            #print(all_data[items])
            a = np.array(all_data[items])
            a = a.flatten()
            if i == 0:
                input_arrays = np.array(a)
            else:
                input_arrays = np.vstack((input_arrays, a))

        ##### Remove NaNs and replace with the mode #################
        imp = Imputer(missing_values='NaN', strategy='most_frequent', axis = 0)
        imp.fit(input_arrays)
        masked_data = imp.transform(input_arrays)

        ####### Now run the PCA! ####################
        #Scale each unit vector
        scaled_data = scale(masked_data)
        shapes = all_data.only_blue.shape
        n_components = len(scaled_data)
        pca = PCA(n_components = n_components)
        pca.fit(scaled_data)
        PC_grids = {}
        #Reshape the principle components into the original grid shape and add them to a dictionary called PCs
        for i in range(n_components):
            PC_grids['PC' + str(i+1)] = pca.components_[i].reshape(shapes) 

        ##### Check for statistical difference and save csv file ###################################
        # Create a bounding box from the locations specified above
        box = shapely.geometry.box(names.minlon[num], names.minlat[num], names.maxlon[num], names.maxlat[num], ccw = True)
        # Only get the polygons that intersect the bounding box (i.e. remove all the irrelevant ones)
        filtered = shp.where(shp.intersects(box)).dropna()
        # Combine all of the relevant polygons into a single polygon
        shp_union = shapely.ops.unary_union(filtered.geometry)
        wanted_keys = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6']
        Only6 = dict((k, PC_grids[k]) for k in wanted_keys if k in PC_grids)
        for idx, item in enumerate(Only6):
            data = PC_grids[item]
            print(item, idx)
            OUTPUT = '/g/data/p25/cek156/' + site + '_PCA_stats.csv'

            # Check for the geobox attribute. If it's not there, apply it from the datacube.
            if not hasattr(data, 'geobox'):
                query = {'lat': (names.maxlat[num], names.minlat[num]), 
                     'lon': (names.minlon[num], names.maxlon[num]),
                     'resolution': (-250, 250), 'output_crs': 'EPSG:3577'}
                elev = dc.load(product = 'srtm_dem1sv1_0', group_by='solar_day', **query)
                print('Applying datacube geobox')
            # Create the mask based on our shapefile
            mask = geometry_mask(warp_geometry(shp_union, shp.crs, elev.crs.wkt), elev.geobox, invert=True)
            # Get data only where the mask is 'true'
            data_masked = data[np.where(mask)]
            # Get data only where the mask is 'false'
            data_maskedF = data[np.where(~ mask)]

            ## Now check for statistical significance
            # Create a new numpy array with just the values
            data_masked2 = np.array(data_masked)
            data_maskedF2 = np.array(data_maskedF)
            # Remove nan values
            data_masked_nonan = data_masked2[~np.isnan(data_masked2)]
            data_maskedF_nonan = data_maskedF2[~np.isnan(data_maskedF2)]
            masked_both = [data_masked_nonan,data_maskedF_nonan]
            if data_masked_nonan.any():
            # How many data points are in each of my NDVI lists?
                size = ([len(i) for i in masked_both])
                # Test with a t-test
                stats_ttest, ttest_pval = scipy.stats.ttest_ind(data_masked_nonan,data_maskedF_nonan, equal_var = 'False')
                # Test with a Kolmogorov-Smirnov test 
                # Our null hypothesis that 2 independent samples are drawn from the same continuous distribution
                stats_KS, KS_pval = scipy.stats.ks_2samp(data_masked_nonan,data_maskedF_nonan)

                # Write to csv file
                row = [item, stats_ttest, stats_KS]
                # Write our stats to a csv file so we can compare them later
                # If this is the first site, make a new file, otherwise, append the existing file
                print('writing to csv')
                write_to_csv(OUTPUT, row, idx)
                # Or if there is no data...
            else:
                print('no useful data')
                row = [item, 'nan', 'nan']
                write_to_csv(OUTPUT, row, idx)

        ########## Plot and save ##############################
        #Read in the CSV with the stats
        PC_stats = pandas.read_csv(OUTPUT)
        # Sort the data alphabetically
        PC_stats = PC_stats.sort_values(by = 'name')
        # Setting the positions and width for the bars
        pos = list(range(len(PC_stats.ttest)))
        width = 0.25
        fig = plt.figure()
        ax = fig.add_subplot(111) # Create matplotlib axes
        ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.
        PC_stats.KS_test.plot(kind='bar', color='red', ax=ax, width=width, position=1)
        PC_stats.ttest.plot(kind='bar', color='blue', ax=ax2, width=width, position=0)
        # Set the y axis label
        ax.set_ylabel('KS test statistic', color='red')
        ax2.set_ylabel('t test statistic', color='blue')
        # Set the labels for the x ticks
        ax.set_xticklabels(PC_stats['name'])
        ax2.set_ylim([300, -300])
        ax.set_ylim([1, -1])
        #Let's save the plot
        fig.savefig('/g/data/p25/cek156/' + site + '_PCA_stats.jpg', bbox_inches='tight')

        ##### Save PCA plot with the greatest ttest statistic #######
        max_PCA_val = max(PC_stats.ttest.min(), PC_stats.ttest.max(), key=abs)
        max_PCA = PC_stats.name[PC_stats['ttest'] == max_PCA_val]
        max_PCA = max_PCA.values[0]

        fig, ax = plt.subplots(figsize=(10,10))
        ax.imshow(PC_grids[str(max_PCA)])
        ax.set_title(str(max_PCA))
        fig.savefig('/g/data/p25/cek156/' + site + '_PCA' + max_PCA + '.jpg', bbox_inches='tight')